# Start

In [23]:
import importlib
import os
import sys
import csv
from pathlib import Path
from docx import Document
from typing import Optional

from langchain_core.tools import Tool, StructuredTool
from langchain.agents import create_agent
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage, SystemMessage

from bsm_multi_agents.graph.state import WorkflowState

In [2]:
cwd = Path.cwd()
project_path = cwd.parent.parent
project_path

PosixPath('/Users/yifanli/Github/model_doc_automation/TooTwo_mcp')

# MCP Server

## call_mcp_tool_async

In [3]:
from bsm_multi_agents.agents.mcp_adapter import call_mcp_tool

In [4]:
server_script_path = os.path.join(project_path, "src/bsm_multi_agents/mcp/server.py")
csv_file_path = os.path.join(project_path, "data/input/dummy_options.csv")
output_dir = os.path.join(project_path, "data/cache")
tool_name = "calculate_bsm_to_file"
args = {
    "input_path": csv_file_path,
    "output_dir": output_dir,
}

In [5]:
call_mcp_tool(tool_name, server_script_path, args)

CallToolResult(meta=None, content=[TextContent(type='text', text='Unknown tool: calculate_bsm_to_file', annotations=None, meta=None)], structuredContent=None, isError=True)

## list_mcp_tools_sync

In [6]:
from bsm_multi_agents.agents.mcp_adapter import list_mcp_tools_sync

In [7]:
server_script_path = os.path.join(project_path, "src/bsm_multi_agents/mcp/server.py")

In [8]:
list_mcp_tools_sync(server_script_path)

[Tool(name='calculate_greeks_to_file', title=None, description='\n    Reads the CSV at input_path, calculates greeks, and saves the result to output_dir.\n    Returns the path to the result file.\n    ', inputSchema={'properties': {'input_path': {'title': 'Input Path', 'type': 'string'}, 'output_dir': {'default': './output', 'title': 'Output Dir', 'type': 'string'}}, 'required': ['input_path'], 'title': 'calculate_greeks_to_fileArguments', 'type': 'object'}, outputSchema={'properties': {'result': {'title': 'Result', 'type': 'string'}}, 'required': ['result'], 'title': 'calculate_greeks_to_fileOutput', 'type': 'object'}, icons=None, annotations=None, meta=None),
 Tool(name='validate_greeks_to_file', title=None, description='\n    Validate Greeks for ALL options from CSV data.\n\n    For each option:\n    - Validates: price > 0\n    - Validates: delta in [0,1] for calls, [-1,0] for puts\n    - Validates: gamma >= 0, vega >= 0\n\n    Args:\n        state: InjectedState, state from the wor

## mcp_tool_to_langchain_tool

In [12]:
from bsm_multi_agents.agents.mcp_adapter import list_mcp_tools_sync
server_script_path = os.path.join(project_path, "src/bsm_multi_agents/mcp/server.py")
csv_file_path = os.path.join(project_path, "data/input/dummy_options.csv")
output_dir = os.path.join(project_path, "data/cache")
mcp_tools = list_mcp_tools_sync(server_script_path)
mcp_tool = mcp_tools[0]
print(mcp_tool)

name='calculate_bsm_to_file' title=None description='\n    Reads the CSV at input_path, calculates BSM prices, and saves the result to output_dir.\n    Returns the path to the result file.\n    ' inputSchema={'properties': {'input_path': {'title': 'Input Path', 'type': 'string'}, 'output_dir': {'title': 'Output Dir', 'type': 'string'}}, 'required': ['input_path', 'output_dir'], 'title': 'calculate_bsm_to_fileArguments', 'type': 'object'} outputSchema={'properties': {'result': {'title': 'Result', 'type': 'string'}}, 'required': ['result'], 'title': 'calculate_bsm_to_fileOutput', 'type': 'object'} icons=None annotations=None meta=None


In [13]:
from bsm_multi_agents.agents.mcp_adapter import mcp_tool_to_langchain_tool

In [14]:
test_tool = mcp_tool_to_langchain_tool(mcp_tool, server_script_path)

In [16]:
test_tool.invoke({"input_path": csv_file_path, "output_dir": output_dir})

CallToolResult(meta=None, content=[TextContent(type='text', text='/Users/yifanli/Github/model_doc_automation/data/cache/dummy_options_bsm_results.csv', annotations=None, meta=None)], structuredContent={'result': '/Users/yifanli/Github/model_doc_automation/data/cache/dummy_options_bsm_results.csv'}, isError=False)

# Tools

## calculate_bsm_to_file

In [17]:
csv_file_path = os.path.join(project_path, "data/input/dummy_options.csv")
output_dir = os.path.join(project_path, "data/cache")

In [18]:
from bsm_multi_agents.mcp import pricing_calculator
importlib.reload(pricing_calculator)
from bsm_multi_agents.mcp.pricing_calculator import calculate_bsm_to_file

In [19]:
calculate_bsm_to_file(csv_file_path, output_dir)

/Users/yifanli/Github/model_doc_automation/src/bsm_multi_agents/mcp/pricing_calculator.py:11: RuntimeWarning: invalid value encountered in sqrt
  d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
/Users/yifanli/Github/model_doc_automation/src/bsm_multi_agents/mcp/pricing_calculator.py:12: RuntimeWarning: invalid value encountered in sqrt
  d2 = d1 - sigma * np.sqrt(T)
/Users/yifanli/Github/model_doc_automation/src/bsm_multi_agents/mcp/pricing_calculator.py:11: RuntimeWarning: invalid value encountered in log
  d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))


'/Users/yifanli/Github/model_doc_automation/data/cache/dummy_options_bsm_results.csv'

## calculate_greeks_to_file

In [20]:
csv_file_path = os.path.join(project_path, "data/input/dummy_options.csv")
output_dir = os.path.join(project_path, "data/cache")

In [21]:
from bsm_multi_agents.mcp import pricing_calculator
importlib.reload(pricing_calculator)
from bsm_multi_agents.mcp.pricing_calculator import calculate_greeks_to_file

In [22]:
calculate_greeks_to_file(csv_file_path, output_dir)

'/Users/yifanli/Github/model_doc_automation/data/cache/dummy_options_greeks_results.csv'

## validate_greeks_to_file

In [106]:
greeks_results_path = os.path.join(project_path, "data/cache/dummy_options_greeks_results.csv")
output_dir = os.path.join(project_path, "data/cache")

In [107]:
from bsm_multi_agents.mcp import pricing_validator
importlib.reload(pricing_validator)
from bsm_multi_agents.mcp.pricing_validator import validate_greeks_to_file

In [108]:
validate_greeks_to_file(greeks_results_path, output_dir)

'/Users/yifanli/Github/model_doc_automation/data/cache/dummy_options_greeks_results_validate_results.csv'

## csv_to_summary

In [11]:
csv_file_path = os.path.join(project_path, "data/input/dummy_options.csv")
output_dir = os.path.join(project_path, "data/cache")
max_rows = 20
csv_path = csv_file_path

In [14]:
with open(csv_path, newline="", encoding="utf-8") as f:
    reader = csv.reader(f)
    rows = list(reader)

In [15]:
header = rows[0]
data_rows = rows[1:max_rows + 1]

In [20]:
doc = Document()
doc.add_heading("Black–Scholes–Merton Pricing Results", level=1)
intro = (
    "This document summarizes sample Black–Scholes–Merton (BSM) option pricing "
    "results. The table below contains a subset of rows from the input dataset."
)
doc.add_paragraph(intro)

num_rows = len(data_rows) + 1
num_cols = len(header)
table = doc.add_table(rows=num_rows, cols=num_cols)
table.style = "Table Grid"

hdr_cells = table.rows[0].cells
for j, col_name in enumerate(header):
    hdr_cells[j].text = str(col_name)

for i, row in enumerate(data_rows, start=1):
    row_cells = table.rows[i].cells
    for j, value in enumerate(row):
        row_cells[j].text = "" if value is None else str(value)

explanation = (
    "The columns include the valuation date, spot price S, strike K, time to "
    "maturity T, risk-free rate r, volatility sigma, option type, asset class, "
    "and the resulting BSM_Price. Rows with missing or invalid inputs may yield "
    "missing or unreliable prices and should be handled carefully in a "
    "production workflow."
)
doc.add_paragraph(explanation)
filename = os.path.basename(csv_path).replace(".csv", "_summary.docx")
output_path = os.path.join(output_dir, filename)

doc.save(output_path)
os.path.abspath(output_path)

'/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/data/cache/dummy_options_summary.docx'

In [ ]:
doc.save(output_dir)

## write_report_to_word

# Agent

## pricing_calculator_agent_node

### Inside Node

In [ ]:
from bsm_multi_agents.config.llm_config import get_llm
from bsm_multi_agents.agents.mcp_adapter import list_mcp_tools_sync

In [65]:
server_path = os.path.join(project_path, "src/bsm_multi_agents/mcp/server.py")
csv_file_path = os.path.join(project_path, "data/input/dummy_options.csv")
output_dir = os.path.join(project_path, "data/cache")

In [66]:
state = WorkflowState(
    csv_file_path=csv_file_path, 
    output_dir=output_dir, 
    server_path=server_path
)

In [67]:
errors = state.get("errors", [])
"csv_file_path" not in state or not state["csv_file_path"]
server_path = state.get("server_path")
output_dir = state.get("output_dir")

In [68]:
mcp_tools = list_mcp_tools_sync(server_path)
langchain_tools = [mcp_tool_to_langchain_tool(t, server_path) for t in mcp_tools]

In [69]:
llm = get_llm().bind_tools(langchain_tools)

In [ ]:
system_prompt = (
    "You are a quantitative calculator agent. "
    "You have access to tools specifically for Greeks calculation via an MCP server. "
    "Use the available tools to process the requested data. "
    "If you are confident, you can run all tools in parallel."
)

user_prompt = (
    f"Input CSV File: {state['csv_file_path']}\n"
    f"Output Directory: {output_dir}\n\n"
    "Please calculate the Greeks for the options in the input CSV file. "
    "Save the results to the output directory. "
    "Ensure you call the calculation tools."
)

messages = list(state.get("messages", []))
if not messages:
        messages.append(SystemMessage(content=system_prompt))
        messages.append(HumanMessage(content=user_prompt))

In [71]:
ai_msg = llm.invoke(messages)
messages.append(ai_msg)
state["messages"] = messages

In [74]:
ai_msg.tool_calls

[{'name': 'calculate_greeks_to_file',
  'args': {'input_path': '/Users/yifanli/Github/model_doc_automation/data/input/dummy_options.csv',
   'output_dir': '/Users/yifanli/Github/model_doc_automation/data/cache'},
  'id': '1f1dfc48-3875-4e96-a690-359e6dcc90b2',
  'type': 'tool_call'}]

### Node Level

In [75]:
server_path = os.path.join(project_path, "src/bsm_multi_agents/mcp/server.py")
csv_file_path = os.path.join(project_path, "data/input/dummy_options.csv")
output_dir = os.path.join(project_path, "data/cache")

In [76]:
from bsm_multi_agents.agents import pricing_calculator_node
importlib.reload(pricing_calculator_node)
from bsm_multi_agents.agents.pricing_calculator_node import pricing_calculator_agent_node

In [77]:
initial_state = WorkflowState(
    csv_file_path=csv_file_path, 
    output_dir=output_dir, 
    server_path=server_path
)

In [78]:
state = pricing_calculator_agent_node(initial_state)

In [79]:
state['messages'][-1].tool_calls

[{'name': 'calculate_greeks_to_file',
  'args': {'input_path': '/Users/yifanli/Github/model_doc_automation/data/input/dummy_options.csv',
   'output_dir': '/Users/yifanli/Github/model_doc_automation/data/cache'},
  'id': 'eaf48236-df13-4d37-ab7a-9b7c0c5a8738',
  'type': 'tool_call'}]

## pricing_calculator_tool_node

### Inside Node

In [ ]:
from bsm_multi_agents.config.llm_config import get_llm
from bsm_multi_agents.agents.mcp_adapter import list_mcp_tools_sync

In [80]:
server_path = os.path.join(project_path, "src/bsm_multi_agents/mcp/server.py")
csv_file_path = os.path.join(project_path, "data/input/dummy_options.csv")
output_dir = os.path.join(project_path, "data/cache")

from bsm_multi_agents.agents import pricing_calculator_node
importlib.reload(pricing_calculator_node)
from bsm_multi_agents.agents.pricing_calculator_node import pricing_calculator_agent_node

initial_state = WorkflowState(
    csv_file_path=csv_file_path, 
    output_dir=output_dir, 
    server_path=server_path
)

state = pricing_calculator_agent_node(initial_state)

In [81]:
from bsm_multi_agents.agents.mcp_adapter import (
    call_mcp_tool
)
from bsm_multi_agents.agents.utils import extract_mcp_content

In [82]:
errors = state.get("errors", [])
messages = list(state.get("messages", []))

In [83]:
last_msg = messages[-1]
not hasattr(last_msg, "tool_calls") or not last_msg.tool_calls

False

In [84]:
server_path = state.get("server_path")

In [85]:
"tool_outputs" not in state or state["tool_outputs"] is None

True

In [86]:
tool_outputs_msgs = []

In [90]:
last_msg.tool_calls

[{'name': 'calculate_greeks_to_file',
  'args': {'input_path': '/Users/yifanli/Github/model_doc_automation/data/input/dummy_options.csv',
   'output_dir': '/Users/yifanli/Github/model_doc_automation/data/cache'},
  'id': '535b3148-714e-4e8d-9641-31b12403dc34',
  'type': 'tool_call'}]

In [ ]:
tool_call = last_msg.tool_calls[0]
tool_name = tool_call["name"]
args = tool_call["args"]
call_id = tool_call["id"]

In [ ]:
raw_result = call_mcp_tool(tool_name, server_path, args)
result_text = extract_mcp_content(raw_result)
result_text

In [92]:
tool_outputs_msgs.append(ToolMessage(content=result_text, tool_call_id=call_id, name=tool_name))
state["greeks_results_path"] = result_text.strip()

In [56]:
content = getattr(tool_res, "content", None)

In [60]:
content[0].text

'/Users/yifanli/Github/model_doc_automation/data/cache/dummy_options_bsm_results.csv'

### Node Level

In [ ]:
server_path = os.path.join(project_path, "src/bsm_multi_agents/mcp/server.py")
csv_file_path = os.path.join(project_path, "data/input/dummy_options.csv")
output_dir = os.path.join(project_path, "data/cache")

from bsm_multi_agents.agents import pricing_calculator_node
importlib.reload(pricing_calculator_node)
from bsm_multi_agents.agents.pricing_calculator_node import (
    pricing_calculator_agent_node,
    pricing_calculator_tool_node,
)

initial_state = WorkflowState(
    csv_file_path=csv_file_path, 
    output_dir=output_dir, 
    server_path=server_path
)

state = pricing_calculator_agent_node(initial_state)
state = pricing_calculator_tool_node(state)

In [94]:
state

{'csv_file_path': '/Users/yifanli/Github/model_doc_automation/data/input/dummy_options.csv',
 'output_dir': '/Users/yifanli/Github/model_doc_automation/data/cache',
 'server_path': '/Users/yifanli/Github/model_doc_automation/src/bsm_multi_agents/mcp/server.py',
 'messages': [SystemMessage(content='You are a quantitative calculator agent. You have access to tools specifically for Greeks calculation via an MCP server. Use the available tools to process the requested data. You should typically validate the input first, then run calculations. If you are confident, you can run all tools in parallel.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Input CSV File: /Users/yifanli/Github/model_doc_automation/data/input/dummy_options.csv\nOutput Directory: /Users/yifanli/Github/model_doc_automation/data/cache\n\nPlease calculate the Greeks for the options in the input CSV file. Save the results to the output directory. Ensure you call the calculation tools.', additional_kw

## pricing_validator_agent_node

### Inside Node

In [111]:
from bsm_multi_agents.config.llm_config import get_llm
from bsm_multi_agents.agents.mcp_adapter import list_mcp_tools_sync

In [112]:
server_path = os.path.join(project_path, "src/bsm_multi_agents/mcp/server.py")
csv_file_path = os.path.join(project_path, "data/input/dummy_options.csv")
output_dir = os.path.join(project_path, "data/cache")

from bsm_multi_agents.agents import pricing_calculator_node
importlib.reload(pricing_calculator_node)
from bsm_multi_agents.agents.pricing_calculator_node import (
    pricing_calculator_agent_node,
    pricing_calculator_tool_node,
)

initial_state = WorkflowState(
    csv_file_path=csv_file_path, 
    output_dir=output_dir, 
    server_path=server_path
)

state = pricing_calculator_agent_node(initial_state)
state = pricing_calculator_tool_node(initial_state)

In [113]:
errors = state.get("errors", [])
"greeks_results_path" not in state or not state["greeks_results_path"]

False

In [114]:
server_path = state.get("server_path")
output_dir = state.get("output_dir")

In [116]:
mcp_tools = list_mcp_tools_sync(server_path)
langchain_tools = [mcp_tool_to_langchain_tool(t, server_path) for t in mcp_tools]
llm = get_llm().bind_tools(langchain_tools)

In [128]:
system_prompt = (
    "You are a quantitative validator agent. "
    "You have access to tools specifically for validation of greeks via an MCP server. "
    "Use the available tools to process the requested data. "
)

user_prompt = (
    f"Input CSV File: {state['greeks_results_path']}\n"
    f"Output Directory: {output_dir}\n\n"
    "Please validate the Greeks for the options in the input CSV file. "
    "Save the results to the output directory. "
    "Ensure you call the validation tools."
)
messages = list(state.get("messages", []))
messages.append(SystemMessage(content=system_prompt))
messages.append(HumanMessage(content=user_prompt))


In [129]:
ai_msg = llm.invoke(messages)

In [130]:
ai_msg.tool_calls

[{'name': 'validate_greeks_to_file',
  'args': {'input_path': '/Users/yifanli/Github/model_doc_automation/data/cache/dummy_options_greeks_results.csv',
   'output_dir': '/Users/yifanli/Github/model_doc_automation/data/cache'},
  'id': '0dcde941-bf4c-4f8b-8d62-23bd3c6ff652',
  'type': 'tool_call'}]

### Node Level

In [132]:
server_path = os.path.join(project_path, "src/bsm_multi_agents/mcp/server.py")
csv_file_path = os.path.join(project_path, "data/input/dummy_options.csv")
output_dir = os.path.join(project_path, "data/cache")

from bsm_multi_agents.agents import pricing_calculator_node
importlib.reload(pricing_calculator_node)
from bsm_multi_agents.agents.pricing_calculator_node import (
    pricing_calculator_agent_node,
    pricing_calculator_tool_node,
)


from bsm_multi_agents.agents.pricing_validator_node import (
    pricing_validator_agent_node,
)

initial_state = WorkflowState(
    csv_file_path=csv_file_path, 
    output_dir=output_dir, 
    server_path=server_path
)

state = pricing_calculator_agent_node(initial_state)
state = pricing_calculator_tool_node(state)
state = pricing_validator_agent_node(state)
state

{'csv_file_path': '/Users/yifanli/Github/model_doc_automation/data/input/dummy_options.csv',
 'output_dir': '/Users/yifanli/Github/model_doc_automation/data/cache',
 'server_path': '/Users/yifanli/Github/model_doc_automation/src/bsm_multi_agents/mcp/server.py',
 'messages': [SystemMessage(content='You are a quantitative calculator agent. You have access to tools specifically for Greeks calculation via an MCP server. Use the available tools to process the requested data. If you are confident, you can run all tools in parallel.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Input CSV File: /Users/yifanli/Github/model_doc_automation/data/input/dummy_options.csv\nOutput Directory: /Users/yifanli/Github/model_doc_automation/data/cache\n\nPlease calculate the Greeks for the options in the input CSV file. Save the results to the output directory. Ensure you call the calculation tools.', additional_kwargs={}, response_metadata={}),
  AIMessage(content='', additional_kwa

In [133]:
state['messages'][-1].tool_calls

[{'name': 'validate_greeks_to_file',
  'args': {'input_path': '/Users/yifanli/Github/model_doc_automation/data/cache/dummy_options_greeks_results.csv',
   'output_dir': '/Users/yifanli/Github/model_doc_automation/data/cache'},
  'id': '8769bdd4-9c07-4bda-9c0b-cf82d61dff0a',
  'type': 'tool_call'}]

## pricing_validator_tool_node

### Inside Node

### Node Level

In [28]:
server_path = os.path.join(project_path, "src/bsm_multi_agents/mcp/server.py")
csv_file_path = os.path.join(project_path, "data/input/dummy_options.csv")
output_dir = os.path.join(project_path, "data/cache")

from bsm_multi_agents.agents import pricing_calculator_node
importlib.reload(pricing_calculator_node)
from bsm_multi_agents.agents.pricing_calculator_node import (
    pricing_calculator_agent_node,
    pricing_calculator_tool_node,
)


from bsm_multi_agents.agents.pricing_validator_node import (
    pricing_validator_agent_node,
    pricing_validator_tool_node,
)

initial_state = WorkflowState(
    csv_file_path=csv_file_path, 
    output_dir=output_dir, 
    server_path=server_path
)

state = pricing_calculator_agent_node(initial_state)
state = pricing_calculator_tool_node(state)
state = pricing_validator_agent_node(state)
state = pricing_validator_tool_node(state)
state

{'csv_file_path': '/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/data/input/dummy_options.csv',
 'output_dir': '/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/data/cache',
 'server_path': '/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/src/bsm_multi_agents/mcp/server.py',
 'messages': [SystemMessage(content='You are a quantitative calculator agent. You have access to tools specifically for Greeks calculation via an MCP server. Use the available tools to process the requested data. If you are confident, you can run all tools in parallel.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Input CSV File: /Users/yifanli/Github/model_doc_automation/TooTwo_mcp/data/input/dummy_options.csv\nOutput Directory: /Users/yifanli/Github/model_doc_automation/TooTwo_mcp/data/cache\n\nPlease calculate the Greeks for the options in the input CSV file. Save the results to the output directory. Ensure you call the calculation tools.', additional_kwargs={}, respon

## report_generator_node

### Inside Node

In [ ]:
server_path = os.path.join(project_path, "src/bsm_multi_agents/mcp/server.py")
csv_file_path = os.path.join(project_path, "data/input/dummy_options.csv")
output_dir = os.path.join(project_path, "data/cache")

from bsm_multi_agents.agents import pricing_calculator_node
importlib.reload(pricing_calculator_node)
from bsm_multi_agents.agents.pricing_calculator_node import (
    pricing_calculator_agent_node,
    pricing_calculator_tool_node,
)


from bsm_multi_agents.agents.pricing_validator_node import (
    pricing_validator_agent_node,
    pricing_validator_tool_node,
)

initial_state = WorkflowState(
    csv_file_path=csv_file_path, 
    output_dir=output_dir, 
    server_path=server_path
)

state = pricing_calculator_agent_node(initial_state)
state = pricing_calculator_tool_node(state)
state = pricing_validator_agent_node(state)
state = pricing_validator_tool_node(state)
state

In [ ]:
state

In [ ]:
errors = state.get("errors", [])
csv_path = state.get("csv_file_path")
summary_docx_path = state.get("summary_docx_path") 

# Graph

In [ ]:
from langgraph.graph import StateGraph, END, START

from bsm_multi_agents.graph.state import WorkflowState
from bsm_multi_agents.agents.pricing_calculator_node import (
    pricing_calculator_agent_node,
    pricing_calculator_tool_node,
)

# from bsm_multi_agents.agents.pricing_validator_node import (
#     pricing_validator_tool_node,
# )



In [102]:
def should_continue_for_pricing_calculator(state):
    messages = state["messages"]
    if not messages:
        return END
        
    last_msg = messages[-1]
    
    # 1. If it's an Agent message (AIMessage) with tool_calls -> Go to Tool
    if hasattr(last_msg, "tool_calls") and last_msg.tool_calls:
        return "pricing_calculator_tool"
    
    # 2. If it's a Tool Execution message (ToolMessage)
    #    Check for errors to decide if we should retry
    if isinstance(last_msg, ToolMessage):
         if last_msg.content.startswith("Error"):
             # OPTIONAL: Check iteration count to prevent infinite loop
             return "pricing_calculator_agent"
         
         # Success -> Finish
         return END
    
    # Default
    return END

In [103]:
graph = StateGraph(WorkflowState)
graph.add_node("pricing_calculator_agent", pricing_calculator_agent_node)
graph.add_node("pricing_calculator_tool", pricing_calculator_tool_node)

graph.add_edge(START, "pricing_calculator_agent")
graph.add_edge("pricing_calculator_agent", "pricing_calculator_tool")
graph.add_conditional_edges(
    "pricing_calculator_tool",
    should_continue_for_pricing_calculator,
    {
        "pricing_calculator_agent": "pricing_calculator_agent", # Retry
        END: END # Success
    }
)
app = graph.compile()

In [104]:
csv_file_path = os.path.join(project_path, "data/input/dummy_options.csv")
output_dir = os.path.join(project_path, "data/cache")
server_path = os.path.join(project_path, "src/bsm_multi_agents/mcp/server.py")
init_state = WorkflowState(
    csv_file_path=csv_file_path,
    output_dir=output_dir,
    server_path=server_path,
    # "remaining_steps": 10,
    # "messages": [HumanMessage(content=f"Load CSV from: {file_path}")],
)

final_state = app.invoke(
    init_state,
    config={"configurable": {"thread_id": "run-1"}}
)

In [105]:
final_state

{'messages': [SystemMessage(content='You are a quantitative calculator agent. You have access to tools specifically for Greeks calculation via an MCP server. Use the available tools to process the requested data. You should typically validate the input first, then run calculations. If you are confident, you can run all tools in parallel.', additional_kwargs={}, response_metadata={}, id='76ae6ab9-4db4-45e3-9924-f96159e1bef0'),
  HumanMessage(content='Input CSV File: /Users/yifanli/Github/model_doc_automation/data/input/dummy_options.csv\nOutput Directory: /Users/yifanli/Github/model_doc_automation/data/cache\n\nPlease calculate the Greeks for the options in the input CSV file. Save the results to the output directory. Ensure you call the calculation tools.', additional_kwargs={}, response_metadata={}, id='5678bbc3-d6a0-4fad-a603-f3a4aed0b84c'),
  AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'qwen3:8b', 'created_at': '2025-12-16T19:27:06.397953Z', 'done': True,